## Basic formula

In [1]:
from collections import namedtuple
import math
from math import sqrt
import numpy as np

import numpy as np
import z3
from z3 import *

def car(real, imag):
    return complex(real,imag)

def pol(rho, phi):
    return (rho, phi)

def car2pol(car):
    x, y = car.real, car.imag
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    phi = phi / 2 / math.pi * 360
    return(rho, phi)

def pol2car(rho, phi):
    phi = phi / 360 * 2 * math.pi
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return complex(x, y)

In [2]:
def checkcar(var):
    if isinstance(var, tuple):
        print(f'        converting {var} into carthesian to calculate with')
        return pol2car(*var)
    return var

def checkpol(var):
    if isinstance(var, complex):
        print(f'        converting {var} into polar to calculate with')
        return car2pol(var)
    if isinstance(var, int):
        print(f'        int found for {var} assuming angle = 0 for polar calculation')
        return (var, 0)
    return var

def parallel(x1,x2):
    return (x1*x2) / (x1 + x2)

def current(voltage, resistance):
    voltage = checkcar(voltage)
    cur = voltage / resistance
    print(f'    current in polar notation: {car2pol(cur)}')
    return cur

def voltage(current, resistance):
    current = checkcar(current)
    vol = current * resistance
    print(f'    voltage in polar notation: {car2pol(vol)}')
    return vol

def deg2rad(deg):
    return deg / 360 * 2 * math.pi

def rad2deg(rad):
    return rad / 2 / math.pi * 360


def Q(voltage, current):
    voltage = checkpol(voltage)
    current = checkpol(current)
    print(f'    Q = {voltage[0]:.2f} * {current[0]:.2f} * sin({voltage[1]:.2f} - {current[1]:.2f}) WARNING, if current enters at - should be -Q')
    return voltage[0] * current[0] * math.sin(deg2rad(voltage[1] - current[1]))
    
def P(voltage, current):
    voltage = checkpol(voltage)
    current = checkpol(current)
    # print(f'    U = {voltage}, I = {current}')
    print(f'    P = {voltage[0]:.2f} * {current[0]:.2f} * cos({voltage[1]:.2f} - {current[1]:.2f}) WARNING, if current enters at - should be -P')
    return voltage[0] * current[0] * math.cos(deg2rad(voltage[1] - current[1]))

def S(P, Q):
    return math.sqrt(P**2 + Q**2)



In [3]:
def capacitor_resistance(capacity, frequency=50):
    return -1 / (2*math.pi * frequency) / capacity

def capacitor_capacity(resistance, frequency=50):
    return 1 / (resistance * 2*math.pi * frequency)
    
def inductor_resistance(capacity, frequency=50):
    return (2*math.pi * frequency) * capacity

def inductor_capacity(resistance, frequency=50):
    return resistance / (2*math.pi*frequency)

## Solver for complex equations

In [4]:
def car2complexexpr(cart):
    return ComplexExpr(cart.real, cart.imag)

def pol2complexexpr(size, angle):
    return car2complexexpr(pol2car(size, angle))

def _to_complex(a):
    if isinstance(a, ComplexExpr):
        return a
    else:
        return ComplexExpr(a, RealVal(0))

def _is_zero(a):
    return (isinstance(a, int) and a == 0) or (is_rational_value(a) and a.numerator_as_long() == 0)

class ComplexExpr:
    def __init__(self, r, i):
        self.r = r
        self.i = i

    def __add__(self, other):
        other = _to_complex(other)
        return ComplexExpr(self.r + other.r, self.i + other.i)

    def __radd__(self, other):
        other = _to_complex(other)
        return ComplexExpr(other.r + self.r, other.i + self.i)

    def __sub__(self, other):
        other = _to_complex(other)
        return ComplexExpr(self.r - other.r, self.i - other.i)

    def __rsub__(self, other):
        other = _to_complex(other)
        return ComplexExpr(other.r - self.r, other.i - self.i)

    def __mul__(self, other):
        other = _to_complex(other)
        return ComplexExpr(self.r*other.r - self.i*other.i, self.r*other.i + self.i*other.r)

    def __mul__(self, other):
        other = _to_complex(other)
        return ComplexExpr(other.r*self.r - other.i*self.i, other.i*self.r + other.r*self.i)

    def inv(self):
        den = self.r*self.r + self.i*self.i
        return ComplexExpr(self.r/den, -self.i/den)

    def __div__(self, other):
        inv_other = _to_complex(other).inv()
        return self.__mul__(inv_other)

    def __rdiv__(self, other):
        other = _to_complex(other)
        return self.inv().__mul__(other)

    def __eq__(self, other):
        other = _to_complex(other)
        return And(self.r == other.r, self.i == other.i)

    def __neq__(self, other):
        return Not(self.__eq__(other))

    def simplify(self):
        return ComplexExpr(simplify(self.r), simplify(self.i))

    def repr_i(self):
        if is_rational_value(self.i):
            return "%s*I" % self.i
        else:
            return "(%s)*I" % str(self.i)

    def __repr__(self):
        if _is_zero(self.i):
            return str(self.r)
        elif _is_zero(self.r):
            return self.repr_i()
        else:
            return "%s + %s" % (self.r, self.repr_i())

def Complex(a):
    return ComplexExpr(Real('%s.r' % a), Real('%s.i' % a))
# I = ComplexExpr(RealVal(0), RealVal(1))

def evaluate_cexpr(m, e):
    return ComplexExpr(m[e.r], m[e.i])

def get_ans(m, var):
    r = evaluate_cexpr(m, var).r
    i = evaluate_cexpr(m, var).i
    real = float(r.numerator_as_long())/float(r.denominator_as_long())
    imag = float(i.numerator_as_long())/float(i.denominator_as_long())
    return complex(real, imag)

### Sandbox

In [5]:
# template for AC nets
I1 = Complex("x")
I2 = Complex("y")
I3 = Complex("z")
s = Tactic('qfnra-nlsat').solver()





# s.add(ComplexExpr(cur.real, cur.imag) - I1 * ComplexExpr(8,4) - I2*ComplexExpr(0,-6) == 0)
# s.add(I2 * ComplexExpr(0,-6) - ComplexExpr(cur2.real, cur2.imag) == 0)
# s.add(I1 + I3 - I2 == 0)

s.add(100-I1*ComplexExpr(0,6.28)-I3*ComplexExpr(0,-6)==0)
s.add(I3*ComplexExpr(0,-6)+I2*10 == 0)
s.add(I1 + I2 -I3 == 0)
s.check()
m = s.model()

for var in [I1,I2, I3]:
    print(car2pol(get_ans(m,var)))
    

(30.864750979992415, -35.21359368159654)
(15.879783371533687, 85.75016285047698)
(26.466305619222812, -4.249837149523023)


In [6]:
# template for DC nets
I1 = Real('x')
I2 = Real('y')
I3 = Real('z')
I4 = Real('r')
I5 = Real('t')

s = Tactic('qfnra-nlsat').solver()
s.add(-10*I1-100-5*I2==0)
s.add(5*I2 + 100 -20*I4 == 0)
s.add(20*I4 + 40 == 0)

s.add(I1 - I2 - I3 == 0)
s.add(I3 - I4 - I5 == 0)
    
s.check()
m = s.model()

m.eval(I1), m.eval(I2), m.eval(I3),m.eval(I4),m.eval(I5)

(4, -28, 32, -2, 34)

## Transformers

In [7]:
Power = namedtuple('Power', 'Watt VAR')

def power(current, resistance):
    if isinstance(resistance, complex):
        return Power(power(current, resistance.real)[0], 
                     power(current, resistance.imag)[0])
    if isinstance(current, complex): # it's in carthesian, convert to polar
        current = car2pol(current)
    if isinstance(current, tuple): # it's in polar, ignore the phase angle
        current = current[0]
    
    p = current ** 2 * resistance
    print(f'    using I2* R: {current}**2 {resistance} = {p}')
    return Power(p,0)


def trans(primaryvoltage, primaryresistance, secondaryresistance, ratio):
    # bring everything to left side
    primaryvoltage = checkcar(primaryvoltage)
    res = primaryresistance + ratio ** 2 * secondaryresistance
    print(f'Total reactance for source {res}')
    Iprim = current(primaryvoltage, res)
    Isec = Iprim * ratio
    
    Uprim = checkcar(primaryvoltage) - primaryresistance * Iprim
    Usec  = Uprim / ratio
    print(f'Transformer: \nIp {car2pol(Iprim)}\nIs {car2pol(Isec)}\nUp {car2pol(Uprim)}\nUs {car2pol(Usec)}')
    print(f'Pprim {P(Uprim, Iprim)}W Qprim {Q(Uprim, Iprim)}VAR')
    psource = -P(primaryvoltage, Iprim)
    qsource = -Q(primaryvoltage, Iprim)
    print(f'Powers on primary side Psource: {psource}, Qsource {qsource}')
    print(f'Resistances on primary side: {power(Iprim, primaryresistance)}')
    print(f'Resistances on secondary side: {power(Isec, secondaryresistance)}')
    print(f'Efficiency transformer: {power(Isec, secondaryresistance).Watt} / {psource} = {power(Isec, secondaryresistance).Watt / psource}')

In [8]:
def nullast(P,U,I):
    S = U * I # schijnbaar vermogen
    Q = (S**2 - P**2)**0.5 # blind vermogen
    # nullasten staan parallel aan de bron dus P = U**2 / R gebruiken
    Rij = U**2 / P  
    Xmu = U**2 / Q
    phi = rad2deg(math.acos((P / (U * I)))) # from formula P = U * I * cos(phi)
    print(
        'Nullast:\n'
        f'S = {S}\n'
        f'Q = {Q} : sqrt(S**2 - P**2) = sqrt({S**2} - {P**2})\n'
        f'phi = {phi} : rad2deg(acos((P / (U * I))))\n'
        f'Rij = {Rij} : U**2 / P\n'
        f'Xmu = {Xmu} : U**2 / Q\n')
    return Rij, Xmu


def kortsluit(P,U,I):
    S = U * I # schijnbaar vermogen
    Q = (S**2 - P**2)**0.5 # blind vermogen
    # nullasten staan in serie met bron dus P = I**2 * R gebruiken
    Rk = P / I**2
    Xk = Q / I**2
    phi = rad2deg(math.acos((P / (U * I)))) # from formula P = U * I * cos(phi)
    print(
        'Kortsluit:\n'
        f'S = {S}\n'
        f'Q = {Q} : sqrt(S**2 - P**2) = sqrt({S**2} - {P**2})\n'
        f'phi = {phi} : rad2deg(acos((P / (U * I))))\n'
        f"Rk = {Rk} : P / I ** 2. Note Rk = R1 + R'2\n"
        f'Xk = {Xk} : Q / I ** 2\n')
    return Rk, Xk



### Sandbox

In [36]:
kortsluit(10,1,2)

ValueError: math domain error

## Asynchronous machines

In [9]:
from math import sqrt
def phase2linevoltage(phasevoltage):
    return sqrt(3) * phasevoltage
def line2phasevoltage(linevoltage):
    return linevoltage / sqrt(3)
line2phasevoltage(10000)

def S3f(Ul, I):
    return sqrt(3) * Ul * I

def Pdriehoek(Pster):
    return 3 * Pster

In [10]:
def slip(fnet, frotor):
    # slip = 1, means rotor is stationary
    # slip = 0, means fnet == frotor
    # slip < 0, means machine is acting as generator
    return (fnet - frotor) / fnet


def slip2freq(slip, fnet=None):
    if not fnet: fnet = 50
    # slip = (fnet - frot) / fnet
    return -(slip * fnet - fnet)


In [11]:
def R2(Rk, ohmmeasured):
    # vervangingsweerstand as berekenen bij bekend zijn van Rk vanuit kortsluitproef
    return Rk - (ohmmeasured / 2)

def Ras(R2, slip):
    # vervangingsweerstand as berekenen inclusief slip
    return R2 * (1-slip) / slip

def Pas(I, R2, slip):
    # asvermogen bepalen
    return 3 * I**2 * Ras(R2, slip)

def koppel(P, freq):
    # P = w T
    return P / (freq * 2 * math.pi)

In [12]:
def Ras2slip(Ras, R2):
    R = Ras / R2
    return 1 / (R + 1)



In [13]:
def asynchrone(voltage, Xk, Rk, R2, slip):
    # gegeven vervangingsschema en een slip bepaal stroom, vermogens en rendement
    # returns asvermogen
    if isinstance(voltage, int):
        voltage = (voltage, 0)
    reactance = complex(Rk + Ras(R2, slip), Xk)
    cur = checkpol(current(voltage, reactance))
    print('Asynchronous machine')
    print('Total reactance = ', reactance)
    print(f'Current = {cur} (voltage / reactance)')
    print('P = ', -3* P(voltage, cur))
    print('Q = ', -3*Q(voltage, cur))
    print('Pas = ', Pas(cur[0], R2, slip))
    print('rendement bij slip > 0 (anders 1/x doen)', Pas(cur[0], R2, slip) / (3 * P(voltage, cur)))
    return Pas(cur[0], R2, slip)

### Sandbox

## Synchnonous machines

In [14]:


# def tas(slip, fnet):
#     was = 2 * math.pi * fnet * (1-slip)
#     print(slip)
#     return p(slip) / was
# s = np.linspace(-0.99,0.99,20)
# fnet = 50
# t = [tas(x, fnet) for x in s]

In [15]:
def X1(Unom, I2nullast, I1kortsluit, I2kortsluit):
    lamb = Unom / I2nullast
    print(f'Bij een I2 van {I2nullast}A wordt de nominale fasespanning gehaald van {Unom}V')
    I1nullast = I1kortsluit / I2kortsluit * I2nullast
    print(f'Bij een I2 van {I2kortsluit}A treedt een I1 op van {I1kortsluit}A, dus bij I2 van {I2nullast}A is I1 {I1nullast}A')
    X1 = Unom / I1nullast
    print('Hieruit volgt:')
    print(f'X1 = {X1} Ohm ({Unom} / {I1nullast})')
    print(f'Lambda = {lamb} ({Unom} / {I2nullast})')
    return X1

X1(230, 4,70, 12)

Bij een I2 van 4A wordt de nominale fasespanning gehaald van 230V
Bij een I2 van 12A treedt een I1 op van 70A, dus bij I2 van 4A is I1 23.333333333333332A
Hieruit volgt:
X1 = 9.857142857142858 Ohm (230 / 23.333333333333332)
Lambda = 57.5 (230 / 4)


9.857142857142858

In [16]:
# capacitive behavior = phase angle current > phase angle voltage, like a balloon, where flow goes before size
# inductive behavior = phase angle voltage > current, like a train where voltage goes before motion

# if I net phase angle < 0: angle in machine diagram goes right up (inductive behavior of net), this is correct since voltage angle > current

In [17]:
def behavior(phi):
    phi = (phi + 360) % 360
    angle_induccapa = 'capacitive' if phi <= 180 else 'inductive'
    angle_motorgenerator = 'generator' if 90 <= phi <= 270 else 'motor'
    return angle_induccapa, angle_motorgenerator

def cosphi_to_options(cosphi):
    # if you don't know if cosphi is for Inet or I1, just enter the cos phi and it will explain how it behaves
    phi = math.acos(cosphi) / 2 / math.pi * 360
    print(f'phi = {phi}, but could also have been {-phi} or with 180 degrees offset if we looked the other way')
    for p in (phi, -phi):
        for reverse in (0,180):
            newphi =  p + reverse
            print(f'{newphi} leads to {behavior(newphi)}')

In [18]:
def syn_net(U1, I1, X1, l = None):
    UX1 = voltage(I1, complex(0,X1)) 
    print(f'Voltage over X1 {car2pol(UX1)}')
    U12 = car2pol(pol2car(*U1) - UX1) # Kirchhoff to determine U12
    print(f'U12 = {U12} = Unet - UX1 --> {U1} - {car2pol(UX1)}')
    print(f'Pas is {3 * P(U12, I1)}')
    QX1 = 3 * I1[0]**2 * X1
    Qas = 3 * Q(U12, I1)
    print(f'Qas = {Qas}')
    print(f'QX1 = {QX1} (3 * I**2 * X1)')
    print(f'Pnet = {-3 * P(U1, I1)}')
    print(f'Qnet = {-3 * Q(U1, I1)}')
    machineQ = QX1 + Qas
    print(QX1, Qas, machineQ)
    induccapa = 'inductive' if machineQ > 0 else 'capacitive'
    print(f'machine is operating {machineQ}, therefore {induccapa}')
    delta = U12[1] - U1[1]
    motorgenerator = 'generator' if delta >= 0 else 'motor'
    print(f'lasthoek delta = {delta}, therefore {motorgenerator} behavior')
    phi = I1[1] % 360
    print(f'asserting behavior phase angle for I1 (phi): {phi}. 0-180=cap, 180-360=ind, 0-90 & 270-360 = motor, 90-270=generator')
    angle_induccapa = 'capacitive' if phi <= 180 else 'inductive'
    angle_motorgenerator = 'generator' if 90 <= phi <= 270 else 'motor'
    assert induccapa == angle_induccapa and motorgenerator == angle_motorgenerator
    
    if l:
        I2 = U12[0] / l
        print(f'I2 is {I2}A')
    return U12


In [19]:
def syn_island(resistance_island, U1, X1, l = None):
    Iz = car2pol(current(U1, resistance_island)) # calculate Iz from U1 and resistance
    I1 = (Iz[0], (Iz[1] + 180) % 360) # I1 == Iz, but reverse phase angle
    print(f'I1 = {I1}')
    
    UX1 = voltage(I1, complex(0,X1)) 
    print(f'Voltage over X1 {car2pol(UX1)}')
    U12 = car2pol(pol2car(*U1) - UX1) # Kirchhoff to determine U12
    print(f'U12 = {U12} = Unet - UX1 --> {U1} - {car2pol(UX1)}')
    print(f'Pas is {3 * P(U12, I1)}')
    QX1 = 3 * I1[0]**2 * X1
    Qas = 3 * Q(U12, I1)
    print(f'Qas = {Qas}')
    print(f'QX1 = {QX1} (3 * I**2 * X1)')
    print(f'Pnet = {-3 * P(U1, I1)}')
    print(f'Qnet = {-3 * Q(U1, I1)}')
    machineQ = QX1 + Qas
    induccapa = 'inductive' if machineQ > 0 else 'capacitive'
    print(f'machine is operating {machineQ}, therefore {induccapa}')
    delta = U12[1] - U1[1]
    motorgenerator = 'generator' if delta >= 0 else 'motor'
    print(f'lasthoek delta = {delta}, therefore {motorgenerator} behavior')
    phi = I1[1] % 360
    print(f'asserting behavior phase angle for I1 (phi): {phi}. 0-180=cap, 180-360=ind, 0-90 & 270-360 = motor, 90-270=generator')
    assert behavior(phi) == (induccapa, motorgenerator) 
    
    if l:
        I2 = U12[0] / l
        print(f'I2 is {I2}A')
    return U12


### Sandbox

In [20]:
syn_net((230,0), (54.35,217), 14)

        converting (54.35, 217) into carthesian to calculate with
    voltage in polar notation: (760.9000000000001, -53.00000000000002)
Voltage over X1 (760.9000000000001, -53.00000000000002)
U12 = (649.0185871656213, 110.55938395147095) = Unet - UX1 --> (230, 0) - (760.9000000000001, -53.00000000000002)
    P = 649.02 * 54.35 * cos(110.56 - 217.00) WARNING, if current enters at - should be -P
Pas is -29950.02958003861
    Q = 649.02 * 54.35 * sin(110.56 - 217.00) WARNING, if current enters at - should be -Q
Qas = -101495.77890926346
QX1 = 124064.745 (3 * I**2 * X1)
    P = 230.00 * 54.35 * cos(0.00 - 217.00) WARNING, if current enters at - should be -P
Pnet = 29950.02958003856
    Q = 230.00 * 54.35 * sin(0.00 - 217.00) WARNING, if current enters at - should be -Q
Qnet = -22568.96609073653
124064.745 -101495.77890926346 22568.966090736532
machine is operating 22568.966090736532, therefore inductive
lasthoek delta = 110.55938395147095, therefore generator behavior
asserting behavior p

(649.0185871656213, 110.55938395147095)

In [21]:
cosphi_to_options(0.8)

phi = 36.86989764584401, but could also have been -36.86989764584401 or with 180 degrees offset if we looked the other way
36.86989764584401 leads to ('capacitive', 'motor')
216.86989764584402 leads to ('inductive', 'generator')
-36.86989764584401 leads to ('inductive', 'motor')
143.13010235415598 leads to ('capacitive', 'generator')


In [22]:
syn_net((6060,0), (6870,143), 0.75)

        converting (6870, 143) into carthesian to calculate with
    voltage in polar notation: (5152.5, -127.00000000000004)
Voltage over X1 (5152.5, -127.00000000000004)
U12 = (10042.618252244087, 24.189172669857317) = Unet - UX1 --> (6060, 0) - (5152.5, -127.00000000000004)
    P = 10042.62 * 6870.00 * cos(24.19 - 143.00) WARNING, if current enters at - should be -P
Pas is -99746859.84417278
    Q = 10042.62 * 6870.00 * sin(24.19 - 143.00) WARNING, if current enters at - should be -Q
Qas = -181357675.22061208
QX1 = 106193025.0 (3 * I**2 * X1)
    P = 6060.00 * 6870.00 * cos(0.00 - 143.00) WARNING, if current enters at - should be -P
Pnet = 99746859.84417269
    Q = 6060.00 * 6870.00 * sin(0.00 - 143.00) WARNING, if current enters at - should be -Q
Qnet = 75164650.22061215
106193025.0 -181357675.22061208 -75164650.22061208
machine is operating -75164650.22061208, therefore capacitive
lasthoek delta = 24.189172669857317, therefore generator behavior
asserting behavior phase angle for 

(10042.618252244087, 24.189172669857317)

In [23]:
syn_net((6060,0), (6870,143), 0.75)

        converting (6870, 143) into carthesian to calculate with
    voltage in polar notation: (5152.5, -127.00000000000004)
Voltage over X1 (5152.5, -127.00000000000004)
U12 = (10042.618252244087, 24.189172669857317) = Unet - UX1 --> (6060, 0) - (5152.5, -127.00000000000004)
    P = 10042.62 * 6870.00 * cos(24.19 - 143.00) WARNING, if current enters at - should be -P
Pas is -99746859.84417278
    Q = 10042.62 * 6870.00 * sin(24.19 - 143.00) WARNING, if current enters at - should be -Q
Qas = -181357675.22061208
QX1 = 106193025.0 (3 * I**2 * X1)
    P = 6060.00 * 6870.00 * cos(0.00 - 143.00) WARNING, if current enters at - should be -P
Pnet = 99746859.84417269
    Q = 6060.00 * 6870.00 * sin(0.00 - 143.00) WARNING, if current enters at - should be -Q
Qnet = 75164650.22061215
106193025.0 -181357675.22061208 -75164650.22061208
machine is operating -75164650.22061208, therefore capacitive
lasthoek delta = 24.189172669857317, therefore generator behavior
asserting behavior phase angle for 

(10042.618252244087, 24.189172669857317)

## Kortsluit

In [24]:
# P = S * cosphi
# Q = S * sinphi


In [25]:
def cosphitosin(cosphi):
    return sqrt(1-cosphi**2)
cosphitosin(1)

0.0

In [26]:
def gen_current(Sn, U3f):
    # can be used for nominal calculations (Un3f) but also for actual calculations (U3f)
    return Sn / (sqrt(3) * U3f)

def gen_reactance(xd, Un, Sn):
    return xd * Un**2 / Sn

def gen_kortsluit(U12, Xg):
    return U12 / Xg

In [27]:
def cable_impedance(R, L, km):
    # assumes R and L are given per km
    X = 314 * L
    return complex(R * km, X * km)
Z = cable_impedance(7.41, 0.32*10**-3, 0.1)

def cable_kortsluit(U, Z):
    return current(U, Z)

def cable_admitantie(C, km):
    return km * 314 * C
cable_admitantie(400*10**-9, 50)

def admitantie_to_reactance(Y):
    Y /= 2
    return complex(0,-1/Y)
    
def Zx(C, km, Rcable, Lcable, Rload, cosphi, Unet = None):
    Y = cable_admitantie(C, km)
    X = admitantie_to_reactance(Y)
    Xcable = cable_impedance(Rcable, Lcable, km)
    Xload = load_impedance(Rload, cosphi)
    print(f'Y {Y}, X {X}, Xcable {Xcable}, Xload {Xload}')
    Z = parallel(Xload, X) + Xcable
    if Unet:
        print(f'I {current(Unet, Z)}')
    return Z



In [37]:
def trans_impedance(uk, U3f, S, P):
    
    Z = uk * U3f**2 / S
    R = P * U3f**2 / S**2
    X = sqrt(Z**2 - R**2)
    return Z, R, X
trans_impedance(25/150, 150*10**3, 60*10**6, 12000)

def trans_kortsluit(Rk, Ifase, Unom3f):
    Uk = Rk * Ifase
    u = Uk / Unom3f / sqrt(3)
    print()
    return u
 
_, a, b = trans_impedance(0.04, 10000, 250000, 4000) 
t = complex(a,b)

In [40]:
trans_impedance(25/150, 150*10**3, 60*10**6, 12000)

(62.5, 0.075, 62.4999549999838)

In [41]:
200*25

5000

In [29]:
def grid_kortsluit(Sk, Un):
    return Sk / (sqrt(3)*Un)
grid_kortsluit(400*10**6, 110000)

def grid_impedance(Un, Sk):
    return Un**2 / Sk / 3 
grid_impedance(63500, 400*10**6/3)

10.080625

In [30]:
def load_impedance(R, cosphi):
    return complex(R * cosphi, R * cosphitosin(cosphi))
Z = load_impedance(1.2, 0.85)
Z *= (10000/400)**2

In [31]:
li = complex(3.25, 1.2)/2
# st = current(10000/sqrt(3), t + l + li)
# v = (voltage(st, l) / (10000/400))
# st = st * (10000/400)

# car2pol(st)

In [32]:
li, t, 

((1.625+0.6j), (6.4+14.664242223858688j))

In [33]:
def vermogens(U3f, impedances):
    
    U1f = U3f / sqrt(3)
    I1f = current(U1f, sum(impedances))
    print('I', car2pol(I1f))
    for i in impedances:
        v = voltage(I1f, i)
        p = 3 * P(v, I1f)
        q = 3 * Q(v, I1f)
        s = S(p,q)
        print(f'P = {p} W\nQ = {q} var\nS = {s} VA')
vermogens(10000,(li,t))

    current in polar notation: (334.7885207524587, -62.267366362537814)
I (334.7885207524587, -62.267366362537814)
    voltage in polar notation: (579.9311277892192, -42.001668891590306)
        converting (430.9615135909112-388.0622202460663j) into polar to calculate with
        converting (155.79258569051146-296.3309364063835j) into polar to calculate with
    P = 579.93 * 334.79 * cos(-42.00 - -62.27) WARNING, if current enters at - should be -P
        converting (430.9615135909112-388.0622202460663j) into polar to calculate with
        converting (155.79258569051146-296.3309364063835j) into polar to calculate with
    Q = 579.93 * 334.79 * sin(-42.00 - -62.27) WARNING, if current enters at - should be -Q
P = 546406.3489346448 W
Q = 201750.03652971506 var
S = 582462.8532325733 VA
    voltage in polar notation: (5356.616332039339, 4.154455159260359)
        converting (5342.541178305346+388.0622202460663j) into polar to calculate with
        converting (155.79258569051146-296.330

In [34]:
v

'PYTHONPATH'

In [35]:
car2pol(v), car2pol(st)

AttributeError: 'str' object has no attribute 'real'

In [ ]:
p = P(v,st)*3
q = Q(v,st)*3
p,q


        converting (226.42305123430185-2.589436427282053j) into polar to calculate with
        converting (159.24626939042764-101.23063703053481j) into polar to calculate with
    P = 226.44 * 188.70 * cos(-0.66 - -32.44) WARNING, if current enters at - should be -P
        converting (226.42305123430185-2.589436427282053j) into polar to calculate with
        converting (159.24626939042764-101.23063703053481j) into polar to calculate with
    Q = 226.44 * 188.70 * sin(-0.66 - -32.44) WARNING, if current enters at - should be -Q


(108957.46953643221, 67525.77487193234)

### Sandbox

## Koppellijn

In [ ]:
def line_power(U1, U2, Xlijn, delta):
    return U1 * U2 / Xlijn * math.sin(delta / 360 * 2 * math.pi)
line_power(400,400,1.5,-40)

def line_current(U1, U2, Xlijn, delta):
    p = line_power(U1, U2, Xlijn, delta)
    print(p)
    return p / (U1 * math.cos(deg2rad(delta/2)))
    
line_current(400,400,1.5,-40)

-68564.01169989753


-182.4107431070233

In [ ]:
def koppellijn(U2, I, angle, X): # U1, P1, P2, Q1, Q2
    Ux = voltage((I, angle), complex(0,X))
    print(f'Ux = {car2pol(Ux)}')
    U1 = pol2car(U2,0) + Ux
    print(f'U1 = {car2pol(U1)} --> Ux + U1')
    P1 = -P(U1, (I, angle))
    print(f'P1 = {P1}')
    P2 = P((U2,0), (I, angle))
    print(f'P2 = {P2}')
    Q1 = -Q(U1, (I, angle))
    print(f'Q1 = {Q1}')
    Q2 = Q((U2,0), (I, angle))
    print(f'Q2 = {Q2}')
    Qx = Q(Ux, (I, angle))
    print(f'Qx = {Qx}')


        converting (50, 30) into carthesian to calculate with
    voltage in polar notation: (75.0, 120.0)
Ux = (75.0, 120.0)
U1 = (467.0385423067351, 7.994140685006319) --> Ux + U1
        converting (462.5+64.9519052838329j) into polar to calculate with
    P = 467.04 * 50.00 * cos(7.99 - 30.00) WARNING, if current enters at - should be -P
P1 = -21650.635094610967
    P = 500.00 * 50.00 * cos(0.00 - 30.00) WARNING, if current enters at - should be -P
P2 = 21650.635094610967
        converting (462.5+64.9519052838329j) into polar to calculate with
    Q = 467.04 * 50.00 * sin(7.99 - 30.00) WARNING, if current enters at - should be -Q
Q1 = 8750.0
    Q = 500.00 * 50.00 * sin(0.00 - 30.00) WARNING, if current enters at - should be -Q
Q2 = -12499.999999999998
        converting (-37.49999999999999+64.9519052838329j) into polar to calculate with
    Q = 75.00 * 50.00 * sin(120.00 - 30.00) WARNING, if current enters at - should be -Q
Qx = 3750.0


In [ ]:
current((400,0), complex(0,3))/2

        converting (400, 0) into carthesian to calculate with
    current in polar notation: (133.33333333333334, -90.0)


-66.66666666666667j

In [ ]:
math.cos(deg2rad(20))

0.9396926207859084

In [ ]:
rad2deg(math.atan(135/230))

30.411081267125372

In [ ]:
voltage((180,77),complex(0,1.5))

        converting (180, 77) into carthesian to calculate with
    voltage in polar notation: (270.0, 167.0)


(-263.0799174920135+60.73678467284359j)

In [ ]:
math.atan(135/230)

0.5307734972029069

In [ ]:
71.8/2

35.9

In [ ]:
line_current(230,230,1.5,71.8)

33502.34768376941


179.82085606564203

In [ ]:
koppellijn(230,180, 30, 1.5)

        converting (180, 30) into carthesian to calculate with
    voltage in polar notation: (270.0, 120.0)
Ux = (270.0, 120.0)
U1 = (252.38858928247927, 67.88890305024783) --> Ux + U1
        converting (95.00000000000003+233.82685902179844j) into polar to calculate with
    P = 252.39 * 180.00 * cos(67.89 - 30.00) WARNING, if current enters at - should be -P
P1 = -35853.45171667576
    P = 230.00 * 180.00 * cos(0.00 - 30.00) WARNING, if current enters at - should be -P
P2 = 35853.451716675765
        converting (95.00000000000003+233.82685902179844j) into polar to calculate with
    Q = 252.39 * 180.00 * sin(67.89 - 30.00) WARNING, if current enters at - should be -Q
Q1 = -27900.00000000001
    Q = 230.00 * 180.00 * sin(0.00 - 30.00) WARNING, if current enters at - should be -Q
Q2 = -20699.999999999996
        converting (-134.99999999999997+233.82685902179844j) into polar to calculate with
    Q = 270.00 * 180.00 * sin(120.00 - 30.00) WARNING, if current enters at - should be -Q
Qx

### Sandbox

## Primaire regeling

In [ ]:
def freq_change(delta_p, knet):
    return -delta_p / knet

def primaire_regeling(freq_change, kgen):
    return -kgen * freq_change

In [ ]:
def secundaire_regeling(p_werkelijk, p_programma, ka, freq_change):
    return (p_werkelijk - p_programma) / ka + freq_change

    

### Sandbox

In [ ]:
df = freq_change(200, 2000)


In [ ]:
4* primaire_regeling(df, 200), 4 * primaire_regeling(df, 300)

(80.0, 120.0)

In [ ]:

secundaire_regeling(-320,-200,800,df), secundaire_regeling(320,200,1200,df)

(-0.25, 0.0)